In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [3]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [4]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [5]:
dimension = 2

In [6]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [7]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [8]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    # v = torch.ones(u_hat.shape)
    # ux = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = v, create_graph = True)[0]
    
    uxx = torch.zeros(x.size()[0], dimension)
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension)
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xb1 = torch.rand(Nb, dimension)
    xb1[:, 0] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension)
    xb2[:, 0] = torch.ones(Nb)
    xb3 = torch.rand(Nb, dimension)
    xb3[:, 1] = torch.zeros(Nb)
    xb4 = torch.rand(Nb, dimension)
    xb4[:, 1] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension)
    dx1[:, 0] = torch.ones(xb1.size()[0])
    uxb1 = (((model(xb1+step_size*dx1) - model(xb1-step_size*dx1))/step_size/2 * (1.0) - 0) + model(xb1) - g(xb1))**2
    uxb2 = (((model(xb2+step_size*dx1) - model(xb2-step_size*dx1))/step_size/2 * (-1.0) - 0) + model(xb2) - g(xb2))**2
    dx2 = torch.zeros(xb1.size()[0], dimension)
    dx2[:, 1] = torch.ones(xb1.size()[0])
    uxb3 = (((model(xb3+step_size*dx2) - model(xb3-step_size*dx2))/step_size/2 * (1.0) - 0) + model(xb3) - g(xb3))**2
    uxb4 = (((model(xb4+step_size*dx2) - model(xb4-step_size*dx2))/step_size/2 * (-1.0) - 0) + model(xb4) - g(xb4))**2
    part_2 = torch.sum((uxb1 + uxb2 + uxb3 + uxb4)[:,0])/xb1.size()[0]
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 4

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 4)
model = initparam(model, 0.5)

In [13]:
import torch.optim as optim
import torch.nn as nn
import time

In [14]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [15]:
traintime = 10000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [16]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_2D_Robin.npy", error_save)
np.save("DGM_relative_error_2D_Robin.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(523.7870)
current relative error is:  tensor(0.8994)
current epoch is:  50
current loss is:  tensor(499.7490)
current relative error is:  tensor(0.7968)
current epoch is:  100
current loss is:  tensor(484.1863)
current relative error is:  tensor(0.6679)
current epoch is:  150
current loss is:  tensor(431.4923)
current relative error is:  tensor(0.4761)
current epoch is:  200
current loss is:  tensor(269.0971)
current relative error is:  tensor(0.4696)
current epoch is:  250
current loss is:  tensor(197.6837)
current relative error is:  tensor(0.6319)
current epoch is:  300
current loss is:  tensor(181.6787)
current relative error is:  tensor(0.6579)
current epoch is:  350
current loss is:  tensor(174.5711)
current relative error is:  tensor(0.7174)
current epoch is:  400
current loss is:  tensor(158.0403)
current relative error is:  tensor(0.7791)
current epoch is:  450
current loss is:  tensor(143.6257)
current relative error is:  tensor(0

current epoch is:  4100
current loss is:  tensor(0.1087)
current relative error is:  tensor(0.0047)
current epoch is:  4150
current loss is:  tensor(0.1036)
current relative error is:  tensor(0.0043)
current epoch is:  4200
current loss is:  tensor(0.1080)
current relative error is:  tensor(0.0054)
current epoch is:  4250
current loss is:  tensor(0.1032)
current relative error is:  tensor(0.0055)
current epoch is:  4300
current loss is:  tensor(0.0911)
current relative error is:  tensor(0.0047)
current epoch is:  4350
current loss is:  tensor(0.0978)
current relative error is:  tensor(0.0041)
current epoch is:  4400
current loss is:  tensor(0.0961)
current relative error is:  tensor(0.0020)
current epoch is:  4450
current loss is:  tensor(0.0881)
current relative error is:  tensor(0.0044)
current epoch is:  4500
current loss is:  tensor(0.0847)
current relative error is:  tensor(0.0036)
current epoch is:  4550
current loss is:  tensor(0.0828)
current relative error is:  tensor(0.0031)


current epoch is:  8200
current loss is:  tensor(0.0185)
current relative error is:  tensor(0.0012)
current epoch is:  8250
current loss is:  tensor(0.0190)
current relative error is:  tensor(0.0013)
current epoch is:  8300
current loss is:  tensor(0.0211)
current relative error is:  tensor(0.0033)
current epoch is:  8350
current loss is:  tensor(0.0175)
current relative error is:  tensor(0.0011)
current epoch is:  8400
current loss is:  tensor(0.0194)
current relative error is:  tensor(0.0014)
current epoch is:  8450
current loss is:  tensor(0.0187)
current relative error is:  tensor(0.0012)
current epoch is:  8500
current loss is:  tensor(0.0166)
current relative error is:  tensor(0.0013)
current epoch is:  8550
current loss is:  tensor(0.0150)
current relative error is:  tensor(0.0010)
current epoch is:  8600
current loss is:  tensor(0.0171)
current relative error is:  tensor(0.0012)
current epoch is:  8650
current loss is:  tensor(0.0166)
current relative error is:  tensor(0.0015)


In [17]:
x_test = torch.rand(2, 10)
torch.abs(model(x) - u_ex(x))

tensor([[2.5813e-04],
        [6.8091e-06],
        [1.3897e-03],
        ...,
        [8.5505e-06],
        [4.3867e-04],
        [3.0858e-03]], grad_fn=<AbsBackward>)